In [ ]:
! pip install mysql-connector
! pip install pyyaml

In [ ]:
import yaml

with open('/tmp/parameters.yml') as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    parameter_value = yaml.load(file, Loader=yaml.FullLoader)

In [ ]:
from datetime import datetime

print("Notebook run on {}".format(datetime.now().strftime("%Y-%m-%d %H:%M:%S")))

In [ ]:
input_bucket = parameter_value['s3bucket']
output_bucket = parameter_value['s3bucket']
output_prefix = parameter_value['output_prefix']

In [ ]:
rdsendpoint = parameter_value['rdsendpoint']
user1 = parameter_value['rdsuser']
password1 = parameter_value['rdspassword']

In [ ]:
database1 = parameter_value['database']
query = parameter_value['query']

In [ ]:
from io import StringIO

import boto3
import pandas as pd

import csv
import json
import mysql.connector
print(rdsendpoint)
mydb = mysql.connector.connect(
    host=rdsendpoint,
    user=user1,
    password=password1,
    database=database1
)

cur = mydb.cursor()

cur.execute(query)

row_headers=[x[0] for x in cur.description] #this will extract row headers
rv = cur.fetchall()
json_data=[]
for result in rv:
    json_data.append(dict(zip(row_headers,result)))
myresult = json.dumps(json_data)

print(myresult)


s3 = boto3.resource('s3')

    
json_file = database1 + ".json"
print(json_file)
s3.Object(output_bucket, output_prefix+"/"+json_file).put(Body=myresult)            

